In [22]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.19.1


In [ ]:
# nltk.download('conll2002')

In [23]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [24]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [25]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [51]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
#        'word.lower=' + word.lower(), 
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
#         'word.isupper=%s' % word.isupper(),
#        'word.istitle=%s' % word.istitle(),
#        'word.isdigit=%s' % word.isdigit(),
#        'postag=' + postag,
#        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
 #           '-1:word.istitle=%s' % word1.istitle(),
 #           '-1:word.isupper=%s' % word1.isupper(),
 #           '-1:postag=' + postag1,
 #           '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
#            '+1:word.istitle=%s' % word1.istitle(),
#            '+1:word.isupper=%s' % word1.isupper(),
#            '+1:postag=' + postag1,
#            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [52]:
sent2features(train_sents[0])[0]

['bias', 'word[-3:]=rne', 'word[-2:]=ne', 'BOS', '+1:word.lower=(']

In [54]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [55]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 952 ms, sys: 0 ns, total: 952 ms
Wall time: 949 ms


In [56]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    
    # minimum frequency
    'feature.minfreq': 5
})

In [57]:
%%time
trainer.train('conll2002-esp.crfsuite')

CPU times: user 9.17 s, sys: 4 ms, total: 9.17 s
Wall time: 9.17 s


In [58]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

In [59]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ', '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ', '.join(sent2labels(example_sent)))

La Coruña , 23 may ( EFECOM ) .

Predicted: B-LOC, I-LOC, O, O, O, O, B-ORG, O, O
Correct:   B-LOC, I-LOC, O, O, O, O, B-ORG, O, O


In [60]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [61]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 236 ms, sys: 0 ns, total: 236 ms
Wall time: 237 ms


feature 로 해당 단어, 품사를 모두 이용할 때의 성능. minfreq=1 로 설정되었기 때문에 overfitting 일 가능성이 있으며, word[i].lower 를 feature 로 이용하면 단어를 외우는 것

             precision    recall  f1-score   support

      B-LOC       0.78      0.75      0.76      1084
      I-LOC       0.87      0.93      0.90       634
     B-MISC       0.69      0.47      0.56       339
     I-MISC       0.87      0.93      0.90       634
      B-ORG       0.82      0.87      0.84       735
      I-ORG       0.87      0.93      0.90       634
      B-PER       0.61      0.49      0.54       557
      I-PER       0.87      0.93      0.90       634

    avg / total       0.81      0.81      0.80      5251

앞/뒤의 단어 유무만 이용해도 어느 정도의 성능은 충분히 나옴. NER 에서는 앞/뒤 단어가 가장 중요한 hints

In [62]:
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       0.69      0.49      0.58      1084
      I-LOC       0.60      0.47      0.52       325
     B-MISC       0.52      0.20      0.29       339
     I-MISC       0.52      0.36      0.43       557
      B-ORG       0.74      0.55      0.63      1400
      I-ORG       0.71      0.52      0.60      1104
      B-PER       0.83      0.69      0.76       735
      I-PER       0.86      0.86      0.86       634

avg / total       0.71      0.54      0.61      6178



In [63]:
debugger = tagger.info()
weights = debugger.state_features

In [64]:
location_features = {feature:weight for feature, weight in weights.items() if 'LOC' in feature[1]}
print(len(location_features))

655


힌트는 앞 뒤, 단어와 현재 단어의 형태

    ('-1:word.lower=en', 'B-LOC') : 3.543269

스페인어 en 은 영어의 in. in PLACE 형태로 이용됨.

In [65]:
for feature, weight in sorted(location_features.items(), key=lambda x:-x[1])[:50]:
    print('{} : {}'.format(feature, weight))

('-1:word.lower=despejado', 'B-LOC') : 6.919385
('-1:word.lower=efe-cantabria', 'B-LOC') : 6.274558
('word[-3:]=yun', 'B-LOC') : 5.874011
('-1:word.lower=palacio', 'I-LOC') : 5.86573
('-1:word.lower=puente', 'I-LOC') : 5.553516
('-1:word.lower=costa', 'I-LOC') : 5.458388
('-1:word.lower=avenida', 'I-LOC') : 5.372484
('word[-3:]=nón', 'B-LOC') : 5.322154
('word[-3:]=iés', 'B-LOC') : 5.147951
('-1:word.lower=nuboso', 'B-LOC') : 5.10912
('word[-3:]=ael', 'B-LOC') : 4.857369
('-1:word.lower=cantabria', 'B-LOC') : 4.785114
('-1:word.lower=santa', 'I-LOC') : 4.763376
('-1:word.lower=parque', 'I-LOC') : 4.587954
('word[-3:]=kio', 'B-LOC') : 4.379538
('+1:word.lower=cairo', 'B-LOC') : 4.342166
('+1:word.lower=coruña', 'B-LOC') : 4.315112
('+1:word.lower=unido', 'B-LOC') : 3.890058
('word[-3:]=lmo', 'B-LOC') : 3.739574
('-1:word.lower=paseo', 'I-LOC') : 3.709889
('-1:word.lower=bulevar', 'I-LOC') : 3.681638
('-1:word.lower=lluvioso', 'B-LOC') : 3.674013
('word[-3:]=uay', 'B-LOC') : 3.642079
('w

In [74]:
print('total num of words in testset = {}'.format(
    sum((len(sent) for sent in test_sents))))

total num of words in testset = 51533
